In [1]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
import numpy as np
import os
import re
from typing import List, Tuple, Dict, Union
from typing import Iterable as Iter
from typing import Callable
from sklearn.base import BaseEstimator, TransformerMixin

In [35]:
data = pd.read_parquet('../acts_parquet/iob_contratos_idato.parquet')
df2 = pd.read_parquet('../acts_parquet/aviso_licitacao_all_entities_iob.parquet')


In [12]:
df = data[list(data.columns)].loc[(data['tipo_ent'] == 'AVISO_LICITACAO')]

In [22]:
for k, v in dict(df.isnull().sum()).items():
    if  v == len(df):
        df.drop(k, axis=1, inplace=True)

In [27]:
df.columns

Index(['id_ato', 'id_dodf', 'tipo_rel', 'tipo_ent', 'texto', 'processo_gdf',
       'modalidade_licitacao', 'numero_licitacao', 'orgao_licitante',
       'sistema_compras', 'valor_estimado_contratacao',
       'data_abertura_licitacao', 'nome_responsavel', 'tipo_objeto',
       'objeto_licitacao', 'codigo_licitacao_sistema_compras', 'IOB'],
      dtype='object')

In [54]:
df.rename({'texto': 'treated_text', 'processo_gdf': 'PROCESSO',
       'modalidade_licitacao': 'MODALIDADE_LICITACAO', 'numero_licitacao': 'NUM_LICITACAO', 'orgao_licitante': 'ORGAO_LICITANTE',
       'sistema_compras': 'SISTEMA_COMPRAS', 'valor_estimado_contratacao': 'VALOR_ESTIMADO',
       'data_abertura_licitacao': 'DATA_ABERTURA', 'nome_responsavel': 'NOME_RESPONSAVEL', 'tipo_objeto': 'TIPO_OBJ',
       'objeto_licitacao': 'OBJ_LICITACAO', 'codigo_licitacao_sistema_compras': 'CODIGO_SISTEMA_COMPRAS',}, axis=1, inplace=True)

In [55]:
df.columns

Index(['id_ato', 'id_dodf', 'tipo_rel', 'tipo_ent', 'treated_text', 'PROCESSO',
       'MODALIDADE_LICITACAO', 'NUM_LICITACAO', 'ORGAO_LICITANTE',
       'SISTEMA_COMPRAS', 'VALOR_ESTIMADO', 'DATA_ABERTURA',
       'NOME_RESPONSAVEL', 'TIPO_OBJ', 'OBJ_LICITACAO',
       'CODIGO_SISTEMA_COMPRAS', 'IOB'],
      dtype='object')

In [56]:
df1 = df[['ORGAO_LICITANTE', 'NUM_LICITACAO', 'OBJ_LICITACAO',
       'MODALIDADE_LICITACAO', 'PROCESSO', 'DATA_ABERTURA', 'TIPO_OBJ',
       'VALOR_ESTIMADO', 'CODIGO_SISTEMA_COMPRAS', 'SISTEMA_COMPRAS',
       'NOME_RESPONSAVEL', 'treated_text']]

In [58]:
df2['NOME_RESPONSAVEL'] = None

In [59]:
df2.columns

Index(['ORGAO_LICITANTE', 'NUM_LICITACAO', 'OBJ_LICITACAO',
       'MODALIDADE_LICITACAO', 'PROCESSO', 'DATA_ABERTURA', 'TIPO_OBJ',
       'VALOR_ESTIMADO', 'CODIGO_SISTEMA_COMPRAS', 'SISTEMA_COMPRAS',
       'treated_text', 'IOB', 'NOME_RESPONSAVEL'],
      dtype='object')

In [60]:
df2 = df2[['ORGAO_LICITANTE', 'NUM_LICITACAO', 'OBJ_LICITACAO',
       'MODALIDADE_LICITACAO', 'PROCESSO', 'DATA_ABERTURA', 'TIPO_OBJ',
       'VALOR_ESTIMADO', 'CODIGO_SISTEMA_COMPRAS', 'SISTEMA_COMPRAS',
       'NOME_RESPONSAVEL', 'treated_text']]

In [61]:
df_acts = pd.concat([df1, df2])

In [63]:
df_acts = df_acts.reset_index(drop=True)

### IOB

In [66]:
from nltk.tokenize import word_tokenize

DEFAULT_TOKENIZER = nltk.RegexpTokenizer(r"\w+|\$[\d\.\-\\]+|\S+").tokenize
class Tokenizer(TransformerMixin, BaseEstimator):
    """ Class to apply tokenizer to pandas DataFrame.
    """
    def __init__(self, tokenizer=DEFAULT_TOKENIZER):
        self.tokenizer = tokenizer


    def __call__(self, X, **kw_params):
        return self.tokenizer(X, **kw_params)


    def fit(self, X, y=None, **fit_params):
        return self


    def transform(self, X, **kw_params):
        if not isinstance(X, pd.Series):
            print("[preprocess.Tokenizer.transform] TYPE:", type(X))
            print('X:::: ', X)
            X = pd.Series(X)
        return X.map(self)


from sklearn.base import BaseEstimator, TransformerMixin

class IOBifyer(TransformerMixin, BaseEstimator):


    @staticmethod
    def find_entity(row, token, ignore_idx=0,
        tokenizer=DEFAULT_TOKENIZER):
        # TODO: aceitar opção de offset, para não ter tennhum tipo de problema
        """Searches for named entities on columns, except by ignore_idx-columns.

        ignore_idx: int indicating which column has
                    the TEXT where the named entity were extracted from
        """
        for idx, column in enumerate(row.keys()):
            if idx == ignore_idx:
                continue
            if isinstance(row[column], str) and \
                token == tokenizer(row[column])[0]:
                return column

        return None


    @staticmethod
    def generate_IOB_labels(row, idx, tokenizer, dbg={}):
        """[summary]

        Args:
            row ([pd.Series]): [pandas series having act text and entities text]
            idx ([int]): [index such that `row[idx]` has the whole act]
            tokenizer ([Callable]): [function to use to tokenize `row[idx]`]
            dbg (dict, optional): [dictionay for debug purposes]. Defaults to {}.

        Returns:
            [Iter[Iter[str]]]: [matrix of IOB labels]
        """
        labels = []
        entity_started = False
        text = row.iloc[idx]
        for token in tokenizer(text):                         # Itera sobre cada token da anotação do ato.
            if not entity_started:                               # Caso uma entidade ainda n tenha sido identificada nos tokens.
                entity = IOBifyer.find_entity(row, token, idx)                 # Busca o token atual no primeiro token de todos os campos do df.
                if entity is not None:                           # Se foi encontrado o token no inicio de alguma entidade ele inicia a comparação token a token com a entidade.
                    entity_started = True
                    token_index = 1
                    labels.append('B-' + entity)
                else:
                    labels.append('O')
            else:     # Caso uma entidade já tenha sido identificada
                if token_index < len(tokenizer(row[entity])) and \
                    token == tokenizer(row[entity])[token_index]:
                    # Checa se o próximo token pertence à entidade
                    # e se o tamanho da entidade chegou ao fim.
                    labels.append('I-' + entity)
                    # Se a entidade ainda possui tokens e a comparação foi bem
                    # sucedida adicione o label I.
                    token_index += 1
                    if token_index >= len(tokenizer(row[entity])):
                        entity_started = False
                else:
                    # Se o token n for igual ou a entidade chegou ao fim.
                    entity_started = False
                    labels.append('O')
        if labels[0] != 'O':
            dbg['l'] = dbg.get('l', []) + [(row, idx)]

        return labels


    @staticmethod
    def dump_iob(tokens_mat, labels_mat, path='dump.txt',
                            sep=' X X ', sent_sep='\n',):
        """This method dumps the token matrix according its IOB labels.

        For debug purposes, a list of list of pairs (token, label) is returned.
        Args:
            tokens_mat ([Iter[Iter[str]]]): [matrix of strings corresponding to tokens]
            labels_mat ([Iter[Iter[str]]]): [matrix of strings corresponding to IOB labels]
            path (str, optional): [Path to dump text file]. Defaults to 'dump.txt'.
            sep (str, optional): [description]. Defaults to ' X X '.
            sent_sep (str, optional): [description]. Defaults to '\n'.

        Returns:
            [List[LIst[Tuple(str, str)]]]: [list of list of pairs (token, label), as dumped. For debug purposes.]
        """
        dbg_mat = []
        if isinstance(path, Path):
            path = path.as_posix()
        if '/' in path:
            os.makedirs('/'.join(path.split('/')[:-1]), exist_ok=True)

        with open(path, 'w') as fp:
            for tokens_lis, labels_lis in zip(tokens_mat, labels_mat):
                dbg_mat.append([])
                for token, label in zip(tokens_lis, labels_lis):
                    dbg_mat[-1].append((token, label))
                    fp.write(f"{token}{sep}{label}\n")
                fp.write(sent_sep)
        return dbg_mat


    def __init__(self, column='act_column',
        tokenizer=DEFAULT_TOKENIZER):
        self.column = column
        self.tokenizer = tokenizer
        self.dbg = {}


    def fit(self, X=None, y=None, **fit_params):
        return self


    def transform(self, df):
        if not isinstance(df, pd.DataFrame):
            raise TypeError(f"`df` expected to be a pd.DataFrame. Got {type(df)}")
        if df.empty:
            print("[core.preprocess]Warning: empty DataFrame. There won't be ioblabels.")
            return pd.Series()

        idx = self.column if isinstance(self.column, int) else  \
                df.columns.get_loc(self.column)
        labels_row = []
        for index, row in df.iterrows():
            try:
                labels_row.append(
                    IOBifyer.generate_IOB_labels(
                        row, idx, self.tokenizer, self.dbg
                    )
                )
            except Exception as e:
                print("problem iobifyin row:", row)
                raise e
        return pd.Series(labels_row)


In [74]:
df_acts.columns

Index(['ORGAO_LICITANTE', 'NUM_LICITACAO', 'OBJ_LICITACAO',
       'MODALIDADE_LICITACAO', 'PROCESSO', 'DATA_ABERTURA', 'TIPO_OBJ',
       'VALOR_ESTIMADO', 'CODIGO_SISTEMA_COMPRAS', 'SISTEMA_COMPRAS',
       'NOME_RESPONSAVEL', 'treated_text'],
      dtype='object')

In [75]:
data = df_acts[['ORGAO_LICITANTE', 'NUM_LICITACAO', 'OBJ_LICITACAO',
       'MODALIDADE_LICITACAO', 'PROCESSO', 'DATA_ABERTURA',
       'VALOR_ESTIMADO', 'CODIGO_SISTEMA_COMPRAS', 'SISTEMA_COMPRAS',
       'NOME_RESPONSAVEL', 'treated_text']]

In [76]:
iob = IOBifyer(column='treated_text')
r = iob.transform(data)

In [77]:
r

0      [O, O, O, B-MODALIDADE_LICITACAO, I-MODALIDADE...
1      [B-MODALIDADE_LICITACAO, I-MODALIDADE_LICITACA...
2      [O, O, O, O, O, O, O, B-MODALIDADE_LICITACAO, ...
3      [O, O, O, B-MODALIDADE_LICITACAO, I-MODALIDADE...
4      [O, O, O, B-MODALIDADE_LICITACAO, I-MODALIDADE...
                             ...                        
430    [B-MODALIDADE_LICITACAO, O, B-NUM_LICITACAO, I...
431    [B-MODALIDADE_LICITACAO, O, B-NUM_LICITACAO, I...
432    [O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...
433    [O, O, O, O, O, O, B-PROCESSO, I-PROCESSO, O, ...
434    [O, O, O, B-MODALIDADE_LICITACAO, I-MODALIDADE...
Length: 435, dtype: object

In [78]:
data.shape, r.shape

((435, 11), (435,))

In [79]:
data["IOB"] = np.nan

In [80]:
for i in range(len(data)):
    data.loc[i, "IOB"] = ' '.join(r[i])

In [82]:
for i in range(len(data)):
    if len(data['IOB'][i].split()) != len(DEFAULT_TOKENIZER(data['treated_text'][i])):
        print(i)

In [83]:
data.to_parquet('../acts_parquet/lic_435_acts_200x_2018_2020.parquet')